연관된 package 로딩하기.

In [3]:
import argparse
import torch
import torch.nn as nn
from torchvision import datasets, transforms

# 모델생성하기

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear( 28 * 28, 500 )
        self.linear2 = nn.Linear( 500, 500 )
        self.linear3 = nn.Linear( 500, 10 )
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.linear1(x))
        x = self.act(self.linear2(x))
        x = self.linear3(x)
        return x

# 메인 함수 만들기

In [10]:
def main():
    parser = argparse.ArgumentParser()
    args = parser.parse_args("")
    args.batch_size = 128
    args.test_batch_size = 128    
    args.epochs = 2
    args.lr = 0.001

    list_train_loss = []
    list_val_loss = []
    list_epochs = []
    
    train_dataset = datasets.MNIST("../data", train=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])  , download=True)
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [50000, 10000])                    
    test_dataset = datasets.MNIST("../data", train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ]), download=True)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)
    
    
    model = Model()
    print('Number of {} parameters'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam( model.parameters(), lr= args.lr)
    #criterion = nn.MSELoss
    
    # training 작업진행.
    
    for ind in range(args.epochs):
        # Training
        train_loss = 0
        model.train()
        for data, target in train_loader:                        
            data = data.view(-1, 28 * 28 )           
            preds = model( data )            
            loss = criterion( preds, target )            
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = train_loss / len( train_loader)    
        print("train_loss", train_loss )    
        list_train_loss.append( train_loss )
        list_epochs.append(ind)

        #Validation
        val_loss = 0
        model.eval()
        for data, target in val_loader:                        
            data = data.view(-1, 28 * 28 )           
            preds = model( data )            
            loss = criterion( preds, target )            
            val_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()            
        val_loss = val_loss / len( val_loader)    
        print("val_loss", val_loss )    
        list_val_loss.append( train_loss )

        # Test
        correct = 0
        model.eval()        
        optimizer.zero_grad()

        for data , target in test_loader:
            data = data.view(-1, 28 * 28)
            preds = model( data )        
            preds_y = preds.max(1, keepdim=True)[1].squeeze()        
            local_correct_sum = preds_y.eq(target).sum()
            correct += local_correct_sum            
        acc = correct.numpy() / len( test_loader.dataset )    
        print("Epoch: {}, Train Loss: {}, Val Loss: {}, Test Acc: {}%".format(ind, train_loss, 0, acc * 100))        


    print(" training done..")

# 실제 실행하기

In [11]:
if __name__ == "__main__":
    main()

100.1%

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz


113.5%

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz


100.4%

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz


180.4%

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!
Number of 648010 parameters
train_loss 0.24339052728946556
val_loss 0.11984257763133774
Epoch: 0, Train Loss: 0.24339052728946556, Val Loss: 0, Test Acc: 96.5%
train_loss 0.08874947123724938
val_loss 0.08487582062901575
Epoch: 1, Train Loss: 0.08874947123724938, Val Loss: 0, Test Acc: 97.53%
 training done..
